# Importing libraries

In [1]:
import os, glob, json, csv, subprocess, sys, re, operator
from git import *
from subprocess import Popen, PIPE
from os import path
import pandas as pd
import itertools

# Defining repository and directories

In [2]:
userhome = os.path.expanduser('~')
repository = userhome + r'/different-diff/dataset/hbase/'
analyze_dir = userhome + r'/different-diff/analyze/analyze_hbase/'

# Defining function to execute git command

In [3]:
def execute_command(cmd, work_dir):
   #Executes a shell command in a subprocess, waiting until it has completed.
    pipe = subprocess.Popen(cmd, shell=True, cwd=work_dir, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    (out, error) = pipe.communicate()
    return out, error
    pipe.wait()

# Loading the datasets

Specifying column names

In [4]:
colnames = ['bug_id','bugintro_commitID','parent_id','filepath','filename','number_of_buggyline']

In [5]:
diffbugs_hist = pd.read_csv(analyze_dir + '03_file-diff/03_number_of_files/histogram_total_bugline.csv')
diffbugs_hist = diffbugs_hist[colnames]
diffbugs_min = pd.read_csv(analyze_dir + '03_file-diff/03_number_of_files/minimal_total_bugline.csv')
diffbugs_min = diffbugs_min[colnames]
diffbugs_myers = pd.read_csv(analyze_dir + '03_file-diff/03_number_of_files/myers_total_bugline.csv')
diffbugs_myers = diffbugs_myers[colnames]
diffbugs_pat = pd.read_csv(analyze_dir + '03_file-diff/03_number_of_files/patience_total_bugline.csv')
diffbugs_pat = diffbugs_pat[colnames]

In [6]:
diffbugs_hist

,bug_id,bugintro_commitID,parent_id,filepath,filename,number_of_buggyline
0,HBASE-17624,e019961150,b2217d1d8f17a8de1a0cf91b74a7170f5ffc1eed,hbase-server/src/main/java/org/apache/hadoop/h...,AbstractFSWALProvider.java,1
1,HBASE-11258,45883c6bd1,2ab2e5c40e6354012f38bd153a77b53335d9b6e8,src/main/java/org/apache/hadoop/hbase/util/Abs...,AbstractHBaseTool.java,13
2,HBASE-16620,e5d77d095c,44812cf1ed6255649bd0d67b1cfe46940f11fc1a,hbase-server/src/main/java/org/apache/hadoop/h...,AbstractHBaseTool.java,5
3,HBASE-15650,d815211f5a,6930da781b2142ed8a0e6fc67095453dea38a072,hbase-server/src/main/java/org/apache/hadoop/h...,AbstractMemStore.java,2
4,HBASE-3100,af7f5c7d01,516667bc74081244ba5c33304125938172e5776a,src/test/java/org/apache/hadoop/hbase/Abstract...,AbstractMergeTestBase.java,143
5,HBASE-19319,f886716617,5a0881a98b3575d900d483222e2fdfab15159656,hbase-procedure/src/main/java/org/apache/hadoo...,AbstractProcedureScheduler.java,53
6,HBASE-15638,95c1dc93fb,cc237c4b78517a6b370c55c8573b14c9edf683de,hbase-server/src/main/java/org/apache/hadoop/h...,AbstractProtobufLogWriter.java,4
7,HBASE-15638,95c1dc93fb,cc237c4b78517a6b370c55c8573b14c9edf683de,hbase-client/src/main/java/org/apache/hadoop/h...,AbstractRegionServerCallable.java,138
8,HBASE-15638,95c1dc93fb,cc237c4b78517a6b370c55c8573b14c9edf683de,hbase-client/src/main/java/org/apache/hadoop/h...,AbstractRpcClient.java,11
9,HBASE-18946,010012cbcb,11e82de8a1d782be85a776ec08e8cd6a071185bf,hbase-server/src/test/java/org/apache/hadoop/h...,AbstractTestDLS.java,1


In [7]:
diffbugs_hist.iloc[571][3]

'src/main/java/org/apache/hadoop/hbase/regionserver/wal/HLog.java'

# Applying "git blame" for files

Define pattern

In [8]:
pattern1 = re.compile(r'(?P<commit_id>[\^]\w+|\w+)\s+(?P<filename>[^\s]+)\s+\((?P<committer>.*?)\s+(?P<date>\d{4}-\d\d-\d\d)\s+(?P<time>\d\d:\d\d:\d\d).*?(?P<line_number>\b\d+\b)\)\s(?P<code>.*)')
pattern2 = re.compile(r'(?P<commit_id>[\^]\w+|\w+)\s+\((?P<committer>.*?)\s+(?P<date>\d{4}-\d\d-\d\d)\s+(?P<time>\d\d:\d\d:\d\d).*?(?P<line_number>\b\d+\b)\)\s(?P<code>.*)')

Define header of table

In [9]:
header1 = ['commit_id', 'filename', 'committer', 'date', 'time', 'line_number', 'code']
header2 = ['commit_id', 'committer', 'date', 'time', 'line_number', 'code']

In [10]:
#Define directory
blame_dirs = analyze_dir + '04_annotate/01_blame_files/'

# Annotating files for histogram

In [11]:
diffbugs_hist.iloc[0]

bug_id                                                       HBASE-17624
bugintro_commitID                                             e019961150
parent_id                       b2217d1d8f17a8de1a0cf91b74a7170f5ffc1eed
filepath               hbase-server/src/main/java/org/apache/hadoop/h...
filename                                      AbstractFSWALProvider.java
number_of_buggyline                                                    1
Name: 0, dtype: object

In [12]:
diffbugs_hist.iloc[0][2][:10]

'b2217d1d8f'

In [13]:
cc = 0
fname = diffbugs_hist.iloc[cc][4] + "_" + diffbugs_hist.iloc[cc][1] + "-" + diffbugs_hist.iloc[cc][2][:10] + "_" + diffbugs_hist.iloc[cc][0]
fname

'AbstractFSWALProvider.java_e019961150-b2217d1d8f_HBASE-17624'

In [14]:
errorfile_hist = []
for cc in range(0, len(diffbugs_hist)):
    sys.stdout.write("\rApplying git blame: %i" % (cc+1) + " / %i" % (len(diffbugs_hist)))
    sys.stdout.flush()
    
    data = diffbugs_hist.iloc[cc]
    
    parent = data[2]
    filepath = data[3]
    
    bugid = data[0]
    introcid = data[1]
    fn = data[4]
    bugnum = data[5]
    fname = data[4] + "_" + data[1] + "-" + data[2][:10] + "_" + data[0] 
    
    blametxt = analyze_dir + '04_annotate/01_blame_files/blame_histogram/txt/blame_' + fname + '_' + str(cc+1) + '.txt'
    
    #checkout parentID and applying git blame
    checkout = "git checkout -f " + parent
    blame = "git blame " + filepath + " > " + blametxt
    
    execute_command(checkout, repository)
    execute_command(blame, repository)
    
    b = open(blametxt,'r')
    c = b.read().split('\n')

    name = analyze_dir + '04_annotate/01_blame_files/blame_histogram/csv/blame_' + fname + '_' + str(cc+1) + '.csv'
    with open(name, 'w', newline='') as f:
        writer = csv.writer(f)
        try:
            try:
                pattern1.match(c[0]).groups()
                writer.writerow(header1)
                for line in c:
                    writer.writerow(pattern1.match(line).groups())
            except:
                pattern2.match(c[0]).groups()
                writer.writerow(header2)
                for line in c:
                    writer.writerow(pattern2.match(line).groups())
        except:
            if os.stat(blametxt).st_size == 0:
                writer.writerow(header1)
                tmp = [bugid, introcid, parent, fn, filepath, fname + '_' + str(cc+1), bugnum]
                errorfile_hist.append(tmp)
                print (' : ' + fname + '_' + str(cc+1) + ' --> file not found')
            pass

#exit from checkout
excheck = "git checkout -f master"
execute_command(excheck, repository)
print ("\n'git blame' implementation is complete")

Applying git blame: 2562 / 2562
'git blame' implementation is complete


In [15]:
#Save error file into CSV file
with open(analyze_dir + '04_annotate/01_blame_files/blame_histogram/error_files_histogram.csv','w') as errorfile:
    header = ['bug_id','bugintro_commitID','parent_id','filename','filepath','f_name_alias','number_of_buggyline']
    writers = csv.writer(errorfile)
    writers.writerow(header)
    for file in errorfile_hist:
        writers.writerow(file)

# Annotating files for minimal

In [16]:
errorfile_min = []
for cc in range(0, len(diffbugs_min)):
    sys.stdout.write("\rApplying git blame: %i" % (cc+1) + " / %i" % (len(diffbugs_min)))
    sys.stdout.flush()
    
    data = diffbugs_min.iloc[cc]
    
    parent = data[2]
    filepath = data[3]
    
    bugid = data[0]
    introcid = data[1]
    fn = data[4]
    bugnum = data[5]
    fname = data[4] + "_" + data[1] + "-" + data[2][:10] + "_" + data[0] 
    
    blametxt = analyze_dir + '04_annotate/01_blame_files/blame_minimal/txt/blame_' + fname + '_' + str(cc+1) + '.txt'
    
    #checkout parentID and applying git blame
    checkout = "git checkout -f " + parent
    blame = "git blame " + filepath + " > " + blametxt
    
    execute_command(checkout, repository)
    execute_command(blame, repository)
    
    b = open(blametxt,'r')
    c = b.read().split('\n')

    name = analyze_dir + '04_annotate/01_blame_files/blame_minimal/csv/blame_' + fname + '_' + str(cc+1) + '.csv'
    with open(name, 'w', newline='') as f:
        writer = csv.writer(f)
        try:
            try:
                pattern1.match(c[0]).groups()
                writer.writerow(header1)
                for line in c:
                    writer.writerow(pattern1.match(line).groups())
            except:
                pattern2.match(c[0]).groups()
                writer.writerow(header2)
                for line in c:
                    writer.writerow(pattern2.match(line).groups())
        except:
            if os.stat(blametxt).st_size == 0:
                writer.writerow(header1)
                tmp = [bugid, introcid, parent, fn, filepath, fname + '_' + str(cc+1), bugnum]
                errorfile_min.append(tmp)
                print (' : ' + fname + '_' + str(cc+1) + ' --> file not found')
            pass

#exit from checkout
excheck = "git checkout -f master"
execute_command(excheck, repository)
print ("\n'git blame' implementation is complete")

Applying git blame: 2562 / 2562
'git blame' implementation is complete


In [17]:
#Save error file into CSV file
with open(analyze_dir + '04_annotate/01_blame_files/blame_minimal/error_files_minimal.csv','w') as errorfile:
    header = ['bug_id','bugintro_commitID','parent_id','filename','filepath','f_name_alias','number_of_buggyline']
    writers = csv.writer(errorfile)
    writers.writerow(header)
    for file in errorfile_min:
        writers.writerow(file)

# Annotating files for myers

In [18]:
errorfile_myers = []
for cc in range(0, len(diffbugs_myers)):
    sys.stdout.write("\rApplying git blame: %i" % (cc+1) + " / %i" % (len(diffbugs_myers)))
    sys.stdout.flush()
    
    data = diffbugs_myers.iloc[cc]
    
    parent = data[2]
    filepath = data[3]
    
    bugid = data[0]
    introcid = data[1]
    fn = data[4]
    bugnum = data[5]
    fname = data[4] + "_" + data[1] + "-" + data[2][:10] + "_" + data[0] 
    
    blametxt = analyze_dir + '04_annotate/01_blame_files/blame_myers/txt/blame_' + fname + '_' + str(cc+1) + '.txt'
    
    #checkout parentID and applying git blame
    checkout = "git checkout -f " + parent
    blame = "git blame " + filepath + " > " + blametxt
    
    execute_command(checkout, repository)
    execute_command(blame, repository)
    
    b = open(blametxt,'r')
    c = b.read().split('\n')

    name = analyze_dir + '04_annotate/01_blame_files/blame_myers/csv/blame_' + fname + '_' + str(cc+1) + '.csv'
    with open(name, 'w', newline='') as f:
        writer = csv.writer(f)
        try:
            try:
                pattern1.match(c[0]).groups()
                writer.writerow(header1)
                for line in c:
                    writer.writerow(pattern1.match(line).groups())
            except:
                pattern2.match(c[0]).groups()
                writer.writerow(header2)
                for line in c:
                    writer.writerow(pattern2.match(line).groups())
        except:
            if os.stat(blametxt).st_size == 0:
                writer.writerow(header1)
                tmp = [bugid, introcid, parent, fn, filepath, fname + '_' + str(cc+1), bugnum]
                errorfile_myers.append(tmp)
                print (' : ' + fname + '_' + str(cc+1) + ' --> file not found')
            pass

#exit from checkout
excheck = "git checkout -f master"
execute_command(excheck, repository)
print ("\n'git blame' implementation is complete")

Applying git blame: 2562 / 2562
'git blame' implementation is complete


In [19]:
#Save error file into CSV file
with open(analyze_dir + '04_annotate/01_blame_files/blame_myers/error_files_myers.csv','w') as errorfile:
    header = ['bug_id','bugintro_commitID','parent_id','filename','filepath','f_name_alias','number_of_buggyline']
    writers = csv.writer(errorfile)
    writers.writerow(header)
    for file in errorfile_myers:
        writers.writerow(file)

# Annotating files for patience

In [20]:
errorfile_pat = []
for cc in range(0, len(diffbugs_pat)):
    sys.stdout.write("\rApplying git blame: %i" % (cc+1) + " / %i" % (len(diffbugs_pat)))
    sys.stdout.flush()
    
    data = diffbugs_pat.iloc[cc]
    
    parent = data[2]
    filepath = data[3]
    
    bugid = data[0]
    introcid = data[1]
    fn = data[4]
    bugnum = data[5]
    fname = data[4] + "_" + data[1] + "-" + data[2][:10] + "_" + data[0] 
    
    blametxt = analyze_dir + '04_annotate/01_blame_files/blame_patience/txt/blame_' + fname + '_' + str(cc+1) + '.txt'
    
    #checkout parentID and applying git blame
    checkout = "git checkout -f " + parent
    blame = "git blame " + filepath + " > " + blametxt
    
    execute_command(checkout, repository)
    execute_command(blame, repository)
    
    b = open(blametxt,'r')
    c = b.read().split('\n')

    name = analyze_dir + '04_annotate/01_blame_files/blame_patience/csv/blame_' + fname + '_' + str(cc+1) + '.csv'
    with open(name, 'w', newline='') as f:
        writer = csv.writer(f)
        try:
            try:
                pattern1.match(c[0]).groups()
                writer.writerow(header1)
                for line in c:
                    writer.writerow(pattern1.match(line).groups())
            except:
                pattern2.match(c[0]).groups()
                writer.writerow(header2)
                for line in c:
                    writer.writerow(pattern2.match(line).groups())
        except:
            if os.stat(blametxt).st_size == 0:
                writer.writerow(header1)
                tmp = [bugid, introcid, parent, fn, filepath, fname + '_' + str(cc+1), bugnum]
                errorfile_pat.append(tmp)
                print (' : ' + fname + '_' + str(cc+1) + ' --> file not found')
            pass

#exit from checkout
excheck = "git checkout -f master"
execute_command(excheck, repository)
print ("\n'git blame' implementation is complete")

Applying git blame: 2562 / 2562
'git blame' implementation is complete


In [21]:
#Save error file into CSV file
with open(analyze_dir + '04_annotate/01_blame_files/blame_patience/error_files_patience.csv','w') as errorfile:
    header = ['bug_id','bugintro_commitID','parent_id','filename','filepath','f_name_alias','number_of_buggyline']
    writers = csv.writer(errorfile)
    writers.writerow(header)
    for file in errorfile_pat:
        writers.writerow(file)

# Delete the txt files from git blame process (unnecessary for next steps)

In [22]:
folder = ['blame_histogram','blame_minimal','blame_myers','blame_patience']
for z, fold in enumerate(folder):
    c = 1
    for file in glob.iglob(analyze_dir + '04_annotate/01_blame_files/' + fold + '/txt/*', recursive=True):
        sys.stdout.write('\r%i ' %(z+1) + 'Deleting file: %i ' %c)
        sys.stdout.flush()
        c += 1
        try:
            os.remove(file)
        except FileNotFoundError:
            print ("\nError: %s" % (file))
print ('\n')
print ('Deleting files are complete')

4 Deleting file: 2562 

Deleting files are complete
